In [6]:
import sys
sys.path.append("../src")
from bigbrotr import Bigbrotr
from event import Event
from relay import Relay
from relay_metadata import RelayMetadata
import utils
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Utils

In [ ]:
private_key, public_key = utils.generate_nostr_keypair()
e = utils.generate_event(private_key, public_key, 1, [], "test")
assert utils.verify_sig(e['id'], e['pubkey'], e['sig']), "Signature verification failed"
assert utils.calc_event_id(e['pubkey'], e['created_at'], e['kind'], e['tags'], e['content']) == e['id'], "Event ID calculation failed"
e = Event(e['id'], e['pubkey'], e['created_at'], e['kind'], e['tags'], e['content'], e['sig'])
e

# Database

In [7]:
bigbrotr = Bigbrotr(
    host="localhost",
    port=5432,
    user="admin",
    password="admin",  
    dbname="bigbrotr"
)

In [8]:
bigbrotr.connect()

In [9]:
query = """
SELECT DISTINCT relay_url
FROM events_relays
WHERE seen_at >= %s
"""
import time
now = int(time.time()) - 120
df = pd.read_sql(query, bigbrotr.conn, params=(now,))
df

/tmp/ipykernel_4177988/1400664167.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, bigbrotr.conn, params=(now,))


,relay_url
0,wss://nostr.bitcoiner.social
1,wss://yabu.me
2,wss://profiles.nos.social
3,wss://nostr-dev.wellorder.net
4,wss://a.nos.lol
5,wss://prod.mosavi.io/v1/ws
6,wss://relay.nos.social
7,wss://d11n.net
8,wss://194.195.222.47:4848
9,wss://relay.wellorder.net


In [13]:
query = "SELECT COUNT(*) FROM events WHERE kind = 1"
bigbrotr.execute(query)
rows = bigbrotr.fetchall()
print(f"Number of events in the database: {rows[0][0]}")

Number of events in the database: 66423419


In [14]:
query = '''
SELECT MIN(inserted_at) AS first_inserted_at
FROM relays
'''
bigbrotr.execute(query)
rows = bigbrotr.fetchall()
first_inserted_at = rows[0][0]
print(f"First inserted relay at: {first_inserted_at}")

First inserted relay at: 1750150988


In [ ]:
# bigbrotr.conn.autocommit = True
# bigbrotr.execute('VACUUM FULL events;')
# bigbrotr.execute('VACUUM FULL events_relays;')
# bigbrotr.execute('VACUUM FULL relay_metadata;')
# bigbrotr.execute('VACUUM FULL relays;')

In [13]:
bigbrotr.close()